In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pickle
from collections import Counter, OrderedDict
import seaborn as sn
import os

In [2]:
def get_file(path):
    read_list =[]
    filelist = os.listdir(path)
    for filename in filelist:
        # de_path = os.path.join(path, filename)
        # print(de_path)
        # if os.path.isfile(filename):
        if filename.endswith(".pickle"):
            read_list.append(filename)
    return read_list

In [5]:
def read_uncertain(path):
    file = open(path, 'rb')
    G_found = pickle.load(file)
    return G_found

def read_edgelist(path):
    edgelist = np.genfromtxt(path, dtype='int64')
    G=nx.Graph()
    G.add_edges_from(edgelist)
    return G

def print_original_statistic(G):  # statistics of original graph
    degree = [G.degree(n) for n in G.nodes()]
    avg_degree = np.mean(degree)
    max_degree = max(degree)
    degree_var = []
    for i in range(len(degree)):
        var = (degree[i] - avg_degree)**2
        degree_var.append(var)
    d_v = sum(degree_var)/len(degree_var)
    cluster_coef = nx.average_clustering(G)

    # largest_cc = max(nx.connected_components(G), key=len)
    # L_cc = G.subgraph(largest_cc)
    # avg_distance = nx.average_shortest_path_length(L_cc)
    # diameter = nx.diameter(L_cc)

    print('number of nodes in original graph:', G.number_of_nodes())
    print('number of edges in original graph:', G.number_of_edges())
    print('average degree in original graph:', avg_degree)
    print('max degree in original graph:', max_degree)
    print('degree variance in original graph:', d_v)
    print('average clustering coefficient in original graph:', cluster_coef)
    # print('number of nodes of largest component in original graph:', L_cc.number_of_nodes())
    # print('number of edges of largest component in original graph:', L_cc.number_of_edges())
    # print('average distance of largest component in original graph:', avg_distance)
    # print('diameter of largest component in original graph:', diameter)

In [7]:
class evaluation:
    def __init__(self, G_found, N):
        self.G_found = G_found # uncertain graph
        self.N = N  # number of possible world
        # self.G = G # original graph

    def possible_world(self):
        # r_bound = 0.5 *((b-a)/error_bound) * math.log(2/p_failure)
        # print(r_bound)
        possible_w = []
        E_c = self.G_found['E_c']
        p_list = self.G_found['p']
        for n in range(self.N):
            w = {}
            E_w = []  # edge list for possible world
            p_w = []
            for i in range(len(E_c)):
                random = np.random.rand()  
                if random < p_list[i]:
                    E_w.append(E_c[i])
                    p_w.append(p_list[i])
            V = set(list(np.array(E_w).flat))
            w['V'] = V
            w['E_w'] = E_w
            w['p']  = p_w
            possible_w.append(w)
        return possible_w
    
    def number_of_edges(self):
        S_ne = sum(self.G_found['p'])
        return round(S_ne)
    
    def average_degree(self):
        n = len(self.G_found['V'])
        S_ad = 2/n * (sum(self.G_found['p']))
        return S_ad

    def max_degree(self):
        max_d = []
        possible_w = self.possible_world() # generate possible worlds
        for w in possible_w:
            # nodes = w['V']
            edges = w['E_w']
            p = w['p']
            G = nx.Graph()
            for i in range(len(edges)):
                G.add_edge(edges[i][0], edges[i][1])  # create graph using networkx
            degree = [G.degree(n) for n in G.nodes()]
            max_d.append(max(degree))
        md = sum(max_d)/len(max_d)  # average max degree of possible worlds
        return round(md)

    def degree_variance(self):
        d_v = []
        possible_w = self.possible_world() # generate possible worlds
        for w in possible_w:
            # nodes = w['V']
            edges = w['E_w']
            p = w['p']
            d = []  # degree variance list
            G = nx.Graph()
            for i in range(len(edges)):
                G.add_edge(edges[i][0], edges[i][1])  # create graph using networkx        
            degree = [G.degree(n) for n in G.nodes()] # degree list
            n = len(w['V'])
            average_d = 2/n * (sum(w['p']))
            for j in range(len(degree)):
                var = (degree[j] - average_d) ** 2
                d.append(var)
            d_var = sum(d)/len(d)
            d_v.append(d_var)
        dv = sum(d_v)/len(d_v) # average degree variance of possible worlds
        return dv

    def degree_distribution(self, G):
        possible_w = self.possible_world()
        # degree_dis = []
        dis = np.zeros(len(G))
        for w in possible_w:
            nodes = w['V']
            edges = w['E_w']
            G = nx.Graph()
            for i in range(len(edges)):
                G.add_edge(edges[i][0], edges[i][1])  # create graph using networkx
            degree = [G.degree(n) for n in G.nodes()]
            degree_counts = dict(Counter(degree))
            sort_degree = OrderedDict(sorted(degree_counts.items()))
            for i in range(len(nodes)):
                if sort_degree.get(i):
                    dis[i] = dis[i] + sort_degree.get(i) / len(degree)
                else:
                    pass
        dis = dis/len(possible_w)
        ind = np.argpartition(dis, -10)[-10:]
        value = dis[ind]
        return ind, value

    def largest_component(self):
        distance = []
        diam = []
        edge = []
        node = []
        possible_w = self.possible_world()
        for w in possible_w:
            # nodes = w['V']
            edges = w['E_w']
            p = w['p']
            G = nx.Graph()
            for i in range(len(edges)):
                G.add_edge(edges[i][0], edges[i][1])
            largest_cc = max(nx.connected_components(G), key=len)
            L_cc = G.subgraph(largest_cc)
            avg_dis =  nx.average_shortest_path_length(L_cc)
            dia = nx.diameter(L_cc)
            distance.append(avg_dis)
            diam.append(dia)
            edge.append(L_cc.number_of_edges())
            node.append(L_cc.number_of_nodes())
        number_of_edges = sum(edge)/len(edge)
        number_of_nodes = sum(node)/len(node)
        avg_dis = sum(distance)/len(distance)
        d = sum(diam)/len(diam)
        return round(number_of_edges), round(number_of_nodes), avg_dis, d
    
    def cluster_coefficient(self):
        cc = []
        possible_w = self.possible_world()
        for w in possible_w:
            # nodes = w['V']
            edges = w['E_w']
            p = w['p']
            G = nx.Graph()
            for i in range(len(edges)):
                G.add_edge(edges[i][0], edges[i][1])  # create graph using networkx         
            coef = nx.average_clustering(G)
            cc.append(coef)
        cluster_coef = sum(cc)/len(cc)
        return cluster_coef

    def print_uncertain_statistis(self):
        print("========================================================")
        print('number of nodes in uncertain graph:', len(self.G_found['V']))
        print('number of edges in uncertain graph:', self.number_of_edges())
        print('average degree in uncertain graph:', self.average_degree())
        print('max degree in uncertain graph:', self.max_degree())
        print('degree variance in uncertain graph:', self.degree_variance())
        print('average clustering coefficient in uncertain graph:', self.cluster_coefficient())
        # number_of_edges, number_of_nodes, average_distance, diameter = self.largest_component()
        # print('number of nodes of largest component in uncertain graph:', number_of_nodes)
        # print('number of edges of largest component in uncertain graph:', number_of_edges)
        # print('average distance of largest component in uncertain graph:', average_distance)
        # print('diameter of largest component in uncertain graph:', diameter)
        # print("========================================================")

    def plot_degree_dist(self, G):
        ind, value = self.degree_distribution(G)
        degree = [G.degree(n) for n in G.nodes()]
        degree_counts = dict(Counter(degree))
        sort_degree = OrderedDict(sorted(degree_counts.items()))
        value_G = []
        for i in ind:
            if sort_degree.get(i):
                value_G.append(sort_degree.get(i)/len(G))
            else:
                value_G.append(0)

        x = ind
        y1 = value
        y2 = value_G
        
        sn.boxplot(x=x,y=y2,palette='vlag')
        sn.stripplot(x=x, y=y1, size =4,color='red')
        plt.xlabel('Degree')
        plt.ylabel('Fraction of nodes')
        plt.title('Degree Distribution')
        plt.savefig('Degree distribution.png')
        # return degree_dist, sort_degree

In [ ]:
if __name__=='__main__':
    G = read_edgelist('facebook_combined.txt')
    print_original_statistic(G)
    path = os.getcwd()
    pickle_files = get_file(path)
    for pickle in pickle_files:
        G_found = read_uncertain(pickle)
        eval = evaluation(G_found, 100)
        eval.print_uncertain_statistis()
        eval.plot_degree_dist(G)